# LFP Extraction

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob
from collections import defaultdict
import re

In [3]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
import itertools

In [4]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity

## Inputs & Data

Explanation of each input and where it comes from.

In [5]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

In [6]:
BBOX_TO_ANCHOR=(1.5, 0.9)
LOC='upper right'

In [7]:
# variables for LFP extraction
FREQ_MIN=0.5
FREQ_MAX=300
FREQ=60
RESAMPLE_RATE=1000
TRIAL_DURATION=10

In [8]:
INPUT_VARIABLE = 1

TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 3
TIME_WINDOW_STEP = 1.5 

TRIAL_TIME_STAMP_DURATION = 1000*10

In [9]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 2.5cm/s", 1: "2.5 to 5cm/s", 2: "5cm/s to 10cm/s"}

In [10]:
BIN_TO_COLOR = {0: {"baseline": "lightblue", "trial": "blue"}, 1: {"baseline": "lightgreen", "trial": "green"}, 2: {"baseline": "lightcoral", "trial": "red"}}
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}
BIN_TO_VELOCITY = {0: "0 to 5cm/s", 1: "5cm/s to 10cm/s", 2: "10cm/s+"}

In [11]:
NUM_LINES = 3

In [12]:
# Generate colors from the "Blues" colormap
LOSING_COLORS = cm.Oranges(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
WINNING_COLORS = cm.Blues(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
REWARDED_COLORS = cm.Greens(np.linspace(0.5, 1, NUM_LINES))
# Generate colors from the "Blues" colormap
OMISSION_COLORS = cm.Reds(np.linspace(0.5, 1, NUM_LINES))

In [13]:
BASELINE_OUTCOME_TO_COLOR = {'lose': "orange",
 'lose_baseline': LOSING_COLORS[0],
 'omission': "red",
 'omission_baseline': "hotpink",
 'rewarded': "green",
 'rewarded_baseline': REWARDED_COLORS[0],
 'win': "blue",
 'win_baseline': WINNING_COLORS[0]}

In [14]:
COMPETITIVE_OUTCOME_TO_COLOR = {'lose_comp': "orange", 
'lose_non_comp': "yellow",
'omission': "red",
'rewarded': "green",
'win_comp': "blue", 
'win_non_comp': WINNING_COLORS[0]}

In [15]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [16]:
CHANNEL_MAPPING_DF = pd.read_excel("../../channel_mapping.xlsx")
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

TONE_TIMESTAMP_DF = pd.read_excel("../../rce_tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


In [40]:
all_session_dir = ['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_19/20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec'
]

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [18]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [19]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


### Getting the subject IDs from the file name

In [20]:
TONE_TIMESTAMP_DF = pd.read_pickle("./proc/melted_trials_and_video.pkl")


In [21]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset="condition").reset_index(drop=True)

In [22]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",baseline,"[1.4426199366829346, 0.9791696179861392, 0.723...","[0.6049464906054562, 0.8003202428686067, 1.576...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]"
1,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",trial,"[5.37049091429759, 5.196457834281414, 4.916080...","[6.000200627117622, 6.7736199704500875, 3.6055...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 0]"
2,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",baseline,"[0.30868701696333833, 0.28295974779669697, 0.3...","[0.9004001623271887, 0.9399580184978504, 0.975...","[(161543.0, 162543.0), (162543.0, 163543.0), (...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 1]"
3,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",trial,"[5.92396632721846, 6.021571590092062, 6.315228...","[7.90304984461326, 5.389214776732418, 17.22809...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[1, 1, 2, 2, 0, 0, 0, 0, 0, 0]"
4,9209413.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,5730880.0,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,...,"[(5730880.0, 5750880.0), (5750880.0, 5770880.0...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[(286544.0, 287544.0), (287544.0, 288544.0), (...","[(276544.0, 277544.0), (277544.0, 278544.0), (...",baseline,"[12.322893761589675, 12.718067202520714, 12.42...","[6.576149902489789, 4.890034291307031, 1.53587...","[(276544.0, 277544.0), (277544.0, 278544.0), (...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


- Original timestamps are based on ephys recordings at 20kHz. The LFP will be at 1kHz, so we will need to divide all the timestamps by 20

In [23]:
all_trials_df["resampled_index"] = all_trials_df["time_stamp_index"] // 20

In [24]:
all_trials_df["recording_dir"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

- Getting a list of all the subjects through the recording name

In [25]:
all_trials_df["all_subjects"] = all_trials_df["recording_dir"].apply(lambda x: ["{}.{}".format(tup[0],tup[1]) for tup in re.findall(r'(\d+)-(\d+)', x.replace("_", "-"))[1:]])

In [26]:
all_trials_df["all_subjects"].head()

0    [1.1, 1.2]
1    [1.1, 1.2]
2    [1.1, 1.2]
3    [1.1, 1.2]
4    [1.1, 1.2]
Name: all_subjects, dtype: object

- Getting the current subject of the recording through the ending of the recording name file

In [27]:
all_trials_df["subject_info"].head()

0    1-1_t1b3L_box1
1    1-1_t1b3L_box1
2    1-1_t1b3L_box1
3    1-1_t1b3L_box1
4    1-1_t1b3L_box1
Name: subject_info, dtype: object

In [28]:
all_trials_df["current_subject"] = all_trials_df["subject_info"].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2]))

In [29]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",baseline,"[1.4426199366829346, 0.9791696179861392, 0.723...","[0.6049464906054562, 0.8003202428686067, 1.576...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]"
1,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",trial,"[5.37049091429759, 5.196457834281414, 4.916080...","[6.000200627117622, 6.7736199704500875, 3.6055...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 0]"
2,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",baseline,"[0.30868701696333833, 0.28295974779669697, 0.3...","[0.9004001623271887, 0.9399580184978504, 0.975...","[(161543.0, 162543.0), (162543.0, 163543.0), (...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 1]"
3,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",trial,"[5.92396632721846, 6.021571590092062, 6.315228...","[7.90304984461326, 5.389214776732418, 17.22809...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[1, 1, 2, 2, 0, 0, 0, 0, 0, 0]"
4,9209413.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,5730880.0,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,...,"[(5730880.0, 5750880.0), (5750880.0, 5770880.0...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[(286544.0, 287544.0), (287544.0, 288544.0), (...","[(276544.0, 277544.0), (277544.0, 278544.0), (...",baseline,"[12.322893761589675, 12.718067202520714, 12.42...","[6.576149902489789, 4.890034291307031, 1.53587...","[(276544.0, 277544.0), (277544.0, 278544.0), (...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


- Labeling the trial as a winner or loser if the winner matches the subject id or not

In [30]:
all_trials_df["trial_outcome"] = all_trials_df.apply(
    lambda x: "win" if str(x["condition"]).strip() == str(x["current_subject"]) 
             else ("lose" if str(x["condition"]) in x["all_subjects"] 
                   else x["condition"]), axis=1)

In [31]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",baseline,"[1.4426199366829346, 0.9791696179861392, 0.723...","[0.6049464906054562, 0.8003202428686067, 1.576...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]"
1,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",trial,"[5.37049091429759, 5.196457834281414, 4.916080...","[6.000200627117622, 6.7736199704500875, 3.6055...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 0]"
2,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",baseline,"[0.30868701696333833, 0.28295974779669697, 0.3...","[0.9004001623271887, 0.9399580184978504, 0.975...","[(161543.0, 162543.0), (162543.0, 163543.0), (...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 1]"
3,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",trial,"[5.92396632721846, 6.021571590092062, 6.315228...","[7.90304984461326, 5.389214776732418, 17.22809...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[1, 1, 2, 2, 0, 0, 0, 0, 0, 0]"
4,9209413.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,5730880.0,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,...,"[(5730880.0, 5750880.0), (5750880.0, 5770880.0...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[(286544.0, 287544.0), (287544.0, 288544.0), (...","[(276544.0, 277544.0), (277544.0, 278544.0), (...",baseline,"[12.322893761589675, 12.718067202520714, 12.42...","[6.576149902489789, 4.890034291307031, 1.53587...","[(276544.0, 277544.0), (277544.0, 278544.0), (...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [32]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in all_trials_df["competition_closeness"].unique()}

In [33]:
competition_closeness_map

{'Subj 2 blocking Subj 1': 'comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 2 Only': 'non_comp',
 'Subj 1 Only': 'non_comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 2 then Subj 1': 'comp',
 'Close Call': 'comp',
 nan: nan}

In [34]:
all_trials_df["competition_closeness"] = all_trials_df["competition_closeness"].map(competition_closeness_map)

In [35]:
all_trials_df["competition_closeness"]

0      comp
1      comp
2      comp
3      comp
4      comp
       ... 
475     NaN
476     NaN
477     NaN
478     NaN
479     NaN
Name: competition_closeness, Length: 480, dtype: object

In [36]:
all_trials_df["competition_closeness"] = all_trials_df.apply(lambda x: "_".join([str(x["trial_outcome"]), str(x["competition_closeness"])]).strip("nan").strip("_"), axis=1)

### Extracting the LFP

In [37]:
all_trials_df["recording_dir"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2'],
      dtype=object)

In [41]:
all_session_dir

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_19/20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec']

In [42]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in all_session_dir:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, "*.rec")):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id="ECU")
            current_recording = se.read_spikegadgets(recording_path, stream_id="trodes")
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=0.5, freq_max=300)
            current_recording = sp.notch_filter(current_recording, freq=60)
            current_recording = sp.resample(current_recording, resample_rate=1000)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except:
            pass



20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged
20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged
20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged


- Filtering for all trials that have labels

In [43]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [44]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",baseline,"[1.4426199366829346, 0.9791696179861392, 0.723...","[0.6049464906054562, 0.8003202428686067, 1.576...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]"
1,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",trial,"[5.37049091429759, 5.196457834281414, 4.916080...","[6.000200627117622, 6.7736199704500875, 3.6055...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 0]"
2,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",baseline,"[0.30868701696333833, 0.28295974779669697, 0.3...","[0.9004001623271887, 0.9399580184978504, 0.975...","[(161543.0, 162543.0), (162543.0, 163543.0), (...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 1]"
3,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",trial,"[5.92396632721846, 6.021571590092062, 6.315228...","[7.90304984461326, 5.389214776732418, 17.22809...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[1, 1, 2, 2, 0, 0, 0, 0, 0, 0]"
4,9209413.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,5730880.0,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,...,"[(5730880.0, 5750880.0), (5750880.0, 5770880.0...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[(286544.0, 287544.0), (287544.0, 288544.0), (...","[(276544.0, 277544.0), (277544.0, 278544.0), (...",baseline,"[12.322893761589675, 12.718067202520714, 12.42...","[6.576149902489789, 4.890034291307031, 1.53587...","[(276544.0, 277544.0), (277544.0, 278544.0), (...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [45]:
all_trials_df["trial_outcome"].unique()

array(['lose', 'win', 'rewarded', 'omission'], dtype=object)

In [46]:
all_trials_df["trial_or_baseline"].unique()

array(['baseline', 'trial'], dtype=object)

## Combining with channel information

In [48]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


In [49]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,trial_or_baseline,current_thorax_velocity,current_chunked_thorax_velocity,chunked_lfp_index,chunked_ephys_timestamp,binned_current_thorax_velocity
0,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",baseline,"[1.4426199366829346, 0.9791696179861392, 0.723...","[0.6049464906054562, 0.8003202428686067, 1.576...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]"
1,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,"[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[(830879.0, 850879.0), (850879.0, 870879.0), (...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(41543.0, 42543.0), (42543.0, 43543.0), (4354...",trial,"[5.37049091429759, 5.196457834281414, 4.916080...","[6.000200627117622, 6.7736199704500875, 3.6055...","[(51543.0, 52543.0), (52543.0, 53543.0), (5354...","[(1030879.0, 1050879.0), (1050879.0, 1070879.0...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 0]"
2,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",baseline,"[0.30868701696333833, 0.28295974779669697, 0.3...","[0.9004001623271887, 0.9399580184978504, 0.975...","[(161543.0, 162543.0), (162543.0, 163543.0), (...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[0, 0, 0, 0, 0, 0, 0, 2, 1, 1]"
3,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,"[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[(3230878.0, 3250878.0), (3250878.0, 3270878.0...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(161543.0, 162543.0), (162543.0, 163543.0), (...",trial,"[5.92396632721846, 6.021571590092062, 6.315228...","[7.90304984461326, 5.389214776732418, 17.22809...","[(171543.0, 172543.0), (172543.0, 173543.0), (...","[(3430878.0, 3450878.0), (3450878.0, 3470878.0...","[1, 1, 2, 2, 0, 0, 0, 0, 0, 0]"
4,9209413.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,5730880.0,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,...,"[(5730880.0, 5750880.0), (5750880.0, 5770880.0...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[(286544.0, 287544.0), (287544.0, 288544.0), (...","[(276544.0, 277544.0), (277544.0, 278544.0), (...",baseline,"[12.322893761589675, 12.718067202520714, 12.42...","[6.576149902489789, 4.890034291307031, 1.53587...","[(276544.0, 277544.0), (277544.0, 278544.0), (...","[(5530880.0, 5550880.0), (5550880.0, 5570880.0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


- Adding all the brain region to ch information

In [50]:
channel_map_and_all_trials_df = all_trials_df.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

- Linking up all LFP calculations with all the trials

In [51]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [52]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [53]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [54]:
melted_channel_map_and_all_trials_df = channel_map_and_all_trials_df.melt(id_vars=id_cols ,value_vars=brain_region_col, var_name='brain_region', value_name='channel')


In [55]:
melted_channel_map_and_all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel
0,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
1,4509412.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,1030879.0,20230617_115521_standard_comp_to_omission_D1_s...,1029,1.0,1-1_t1b3L_box1,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
2,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
3,6909411.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,3430878.0,20230617_115521_standard_comp_to_omission_D1_s...,3425,1.0,1-1_t1b3L_box1,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
4,9209413.0,1.0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,dio_ECU_Din1,5730880.0,20230617_115521_standard_comp_to_omission_D1_s...,5720,1.0,1-1_t1b3L_box1,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0


In [56]:
melted_channel_map_and_all_trials_df.tail()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel
2395,68781880.0,1.0,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,65588836.0,20230621_111240_standard_comp_to_omission_D5_s...,65550,1.0,1-4_t3b3L_box1,...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
2396,70681909.0,1.0,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,67488865.0,20230621_111240_standard_comp_to_omission_D5_s...,67447,1.0,1-4_t3b3L_box1,...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
2397,70681909.0,1.0,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,67488865.0,20230621_111240_standard_comp_to_omission_D5_s...,67447,1.0,1-4_t3b3L_box1,...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
2398,71681918.0,1.0,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,68488874.0,20230621_111240_standard_comp_to_omission_D5_s...,68444,1.0,1-4_t3b3L_box1,...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
2399,71681918.0,1.0,20230621_111240_standard_comp_to_omission_D5_s...,20230621_111240_standard_comp_to_omission_D5_s...,dio_ECU_Din1,68488874.0,20230621_111240_standard_comp_to_omission_D5_s...,68444,1.0,1-4_t3b3L_box1,...,2,1.4,NaN,15,14,13,12,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0


- Getting the traces for each trial for each brain region

In [57]:
melted_channel_map_and_all_trials_df["channel"] = melted_channel_map_and_all_trials_df["channel"].astype(int).astype(str)
melted_channel_map_and_all_trials_df["resampled_index"] = melted_channel_map_and_all_trials_df["resampled_index"].astype(int)

In [58]:
melted_channel_map_and_all_trials_df["chunked_lfp_index"].iloc[0]

[(41543.0, 42543.0),
 (42543.0, 43543.0),
 (43543.0, 44543.0),
 (44543.0, 45543.0),
 (45543.0, 46543.0),
 (46543.0, 47543.0),
 (47543.0, 48543.0),
 (48543.0, 49543.0),
 (49543.0, 50543.0),
 (50543.0, 51543.0)]

In [59]:
melted_channel_map_and_all_trials_df["binned_current_thorax_velocity"].iloc[0]

[0, 0, 0, 0, 0, 0, 0, 2, 2, 1]

In [61]:
melted_channel_map_and_all_trials_df.columns

Index(['time', 'state', 'recording_dir', 'recording_file', 'din',
       'time_stamp_index', 'video_file', 'video_frame', 'video_number',
       'subject_info', 'condition', 'competition_closeness', 'Unnamed: 13',
       'resampled_index', 'all_subjects', 'current_subject', 'trial_outcome',
       'video_name', 'file_path', 'start_frame', 'stop_frame',
       'individual_subj', 'all_subj', 'recording_name', 'all_sleap_data',
       'locations', 'track_names', 'subject_to_index', 'subject_id',
       'subject_locations', 'corner_path', 'corner_parts', 'bottom_width',
       'top_width', 'right_height', 'left_height', 'average_height',
       'average_width', 'width_ratio', 'height_ratio', 'rescaled_locations',
       'box_top_left', 'box_bottom_left', 'box_bottom_right', 'reward_port',
       'box_top_right', 'all_thorax_locations', 'all_thorax_velocity',
       'trial_thorax_velocity', 'baseline_thorax_velocity',
       'trial_chunked_thorax_velocity', 'baseline_chunked_thorax_velocity

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"] = melted_channel_map_and_all_trials_df.apply(lambda x: 
[x["all_ch_lfp"].get_traces(channel_ids=[x["channel"]], start_frame=chunk_index[0], end_frame=chunk_index[1]).T[0] for chunk_index in x["chunked_lfp_index"]], axis=1)

In [ ]:
len(melted_channel_map_and_all_trials_df["chunked_current_trace"].iloc[0])

In [ ]:
melted_channel_map_and_all_trials_df["chunked_current_trace"]

In [ ]:
melted_channel_map_and_all_trials_df["chunk_id"] = [list(range(0,10))] * len(melted_channel_map_and_all_trials_df)

In [ ]:
exploded_channel_map_and_all_trials_df = melted_channel_map_and_all_trials_df.explode(["current_chunked_thorax_velocity", "chunked_current_trace", "binned_current_thorax_velocity", "chunked_lfp_index",	"chunked_ephys_timestamp", "chunk_id"])

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: len(x))


In [ ]:
exploded_channel_map_and_all_trials_df["len_chunked_current_trace"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: x[:RESAMPLE_RATE])

In [ ]:
exploded_channel_map_and_all_trials_df["chunked_current_trace"]

In [ ]:
# exploded_channel_map_and_all_trials_df.to_pickle("./")

In [ ]:
raise ValueError()

- Calcuating the power at each frequency band

In [ ]:
exploded_channel_map_and_all_trials_df["multitaper"] = exploded_channel_map_and_all_trials_df["chunked_current_trace"].apply(lambda x: Multitaper(time_series=x,                                                                                        sampling_frequency=RESAMPLE_RATE))

In [ ]:
exploded_channel_map_and_all_trials_df["connectivity"] = exploded_channel_map_and_all_trials_df["multitaper"].apply(lambda x: Connectivity.from_multitaper(x))

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.frequencies)

In [ ]:
exploded_channel_map_and_all_trials_df["power"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.power().squeeze())

In [ ]:
exploded_channel_map_and_all_trials_df.head()

In [ ]:
exploded_channel_map_and_all_trials_df.tail()

# Plotting by velocity

- Grouping all the rows(1s segment of trials) that share the same brain region, whether its before or after the tone, and velocity grouping

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_or_baseline', 'binned_current_thorax_velocity']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

- Calculating the mean and sem of the LFP power

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.legend(loc="lower left")
    
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue

    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]

    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    # plt.ylim(0, 0.005)
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} for {}".format(BIN_TO_VELOCITY[row["binned_current_thorax_velocity"]], \
            row["trial_or_baseline"]), color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]],
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)

            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], \
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2,
            color=BIN_TO_COLOR[row["binned_current_thorax_velocity"]][row["trial_or_baseline"]])
        except Exception as e: 
            print(e)
            continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_velocity_baselinetrial/\
    rce_lfp_power_velocity_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))


## Plotting all the lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["trial_outcome"].unique()

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq) 
    # plt.yscale("log")
    # plt.ylim(0.01, 0.06)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend(loc="lower left")
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.1, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_lower_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"] = exploded_channel_map_and_all_trials_df["power"].apply(lambda x: x[4:13])

In [ ]:
exploded_channel_map_and_all_trials_df["theta_power"].iloc[0]

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"] = exploded_channel_map_and_all_trials_df["theta_power"].apply(lambda x: np.mean(x))

In [ ]:
exploded_channel_map_and_all_trials_df["averaged_theta_power"]

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["all_thorax_velocity"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df.info()

In [ ]:
exploded_channel_map_and_all_trials_df.drop(columns=["multitaper", "connectivity", 'all_thorax_locations', 'all_thorax_velocity', 'all_ch_lfp']).to_pickle("./proc/rce_1_sec_interval_lfp_power_spectra_and_velocity.pkl")

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'trial_outcome', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
exploded_channel_map_and_all_trials_df["frequencies"].iloc[0][4:13]

In [ ]:
grouped_all_trials_df["power"].iloc[0].shape

In [ ]:
grouped_all_trials_df["power"].iloc

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)   
    plt.yscale("log")
    plt.ylim(0.01, 0.1)

    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.008) 
    
    for trial_or_baseline, outcome in itertools.product(region_df["trial_or_baseline"].unique(), sorted(region_df["trial_outcome"].unique())[::-1]):
        outcome_df = region_df[(region_df["trial_or_baseline"] == trial_or_baseline) & (region_df["trial_outcome"] == outcome)]
        for index, row in outcome_df.iterrows():
            try:
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
                label="{} {}".format(outcome, trial_or_baseline), color=BASELINE_OUTCOME_TO_COLOR[outcome], \
                linestyle=TRIAL_OR_BASELINE_TO_STYLE[trial_or_baseline], linewidth=3)
                
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
                row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
                alpha=0.2, color=BASELINE_OUTCOME_TO_COLOR[outcome])
            
            except Exception as e: 
                print(e)
                continue
    plt.legend()
    plt.savefig("./proc/velocity_parsing/plot_region_line_trialoutcome_baselinetrial/\
    rce_higher_velocity_lfp_power_trialoutcome_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]))

## Plotting all the competitiveness and lower velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 0]

In [ ]:
speed_filtered_all_trials_df.shape

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'competition_closeness', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
grouped_all_trials_df["competition_closeness"].unique()

# Drawing plots all together

In [ ]:
region_df

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)  
    plt.yscale("log")
    plt.ylim(0.01, 0.1) 


    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()


In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
            
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 0 to 5cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_lower_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

## Plotting all the higher velocities together

In [ ]:
speed_filtered_all_trials_df = exploded_channel_map_and_all_trials_df[exploded_channel_map_and_all_trials_df["binned_current_thorax_velocity"] == 1]

In [ ]:
grouped_all_trials_df = speed_filtered_all_trials_df.groupby(['brain_region', 'competition_closeness', 'trial_or_baseline']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

# Drawing plots all together

In [ ]:
low_freq = 0
high_freq = 12
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.01, 0.25) 

    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 

    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 12
high_freq = 30
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.002, 0.015) 
    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
low_freq = 30
high_freq = 90
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {} at 5cm/s to 10cm/s".format(region.split("_")[-1]))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(low_freq, high_freq)    
    plt.yscale("log")
    plt.ylim(0.0003, 0.005) 
    
    for index, row in region_df.iterrows():
        try:
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], \
            label="{} {}".format(row["competition_closeness"], row["trial_or_baseline"]), color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]], \
            linestyle=TRIAL_OR_BASELINE_TO_STYLE[row["trial_or_baseline"]], linewidth=3)
            
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], 
            row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], \
            alpha=0.1, color=COMPETITIVE_OUTCOME_TO_COLOR[row["competition_closeness"]])
        
        except Exception as e: 
            print(e)
            continue
            
    # Create a legend outside of the plot
    legend = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the plot along with the legend outside of it   
    plt.savefig("./proc/velocity_parsing/plot_region_line_competitioncloseness_baselinetrial/\
    rce_higher_velocity_lfp_power_competitioncloseness_baselinetrial_freq_{}_to_{}_region_{}.png".format(low_freq, high_freq, region.split("_")[-1]), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Display the plot
    plt.show()

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df.columns

In [ ]:
exploded_channel_map_and_all_trials_df["current_thorax_velocity"]

In [ ]:
melted_channel_map_and_all_trials_df["subject_id"].unique()

# Drawing Spectogram

In [ ]:
exploded_channel_map_and_all_trials_df["power"].iloc[0].shape

In [ ]:
exploded_channel_map_and_all_trials_df["brain_region"]

In [ ]:
exploded_channel_map_and_all_trials_df[""]

In [ ]:
grouped_all_trials_df

In [ ]:
raise ValueError()

In [ ]:
exploded_channel_map_and_all_trials_df

In [ ]:
exploded_channel_map_and_all_trials_df["recording_file"].unique()

In [ ]:
exploded_channel_map_and_all_trials_df["subject_id"].unique()